In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()
import os

import argparse
import logging
import os
import sys

from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import torch.optim
from accelerate import Accelerator
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import losses
from sentence_transformers.datasets import NoDuplicatesDataLoader
from sentence_transformers.readers import InputExample
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/cluster/scratch/yakram/conda_env/'

os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [2]:
from pandarallel import pandarallel
import random, ast
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
LANGUAGE = 'german'
model_dir_language = 'de'

data_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/{LANGUAGE}_data/'

In [4]:
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli') # german sentence transformer, en is different

In [5]:
MemConcept_description = pd.read_csv(os.path.join(data_dir, 'query_MemConcept_description.rq'), sep='\t', index_col=[0])
SkillsHier_description = pd.read_csv(os.path.join(data_dir, 'query_SkillsHier_description.rq'), sep='\t', index_col=[0])
MemConcept_prefLabel = pd.read_csv(os.path.join(data_dir, 'query_MemConcept_prefLabel.rq'), sep='\t', index_col=[0])
SkillsHier_prefLabel = pd.read_csv(os.path.join(data_dir, 'query_SkillsHier_prefLabel.rq'), sep='\t', index_col=[0])

concept_schemes = pd.read_csv(os.path.join(data_dir, 'query_concept_schemes_combined.rq'), sep='\t', index_col=[0])

In [6]:
MemConcept_SkillsHier_prefLabel = pd.concat([MemConcept_prefLabel, SkillsHier_prefLabel])
MemConcept_SkillsHier_description = pd.concat([MemConcept_description, SkillsHier_description])

pref_desc_data = MemConcept_SkillsHier_prefLabel.merge(MemConcept_SkillsHier_description, on=['?a'], how='left')

skill_pref_desc = concept_schemes.merge(pref_desc_data, on=['?a'], how='left')
skill_pref_desc.dropna(inplace=True)

skill_pref_desc.shape

(13921, 3)

### Check statistics of term-desc intersections in matching pairs

In [7]:
from nltk.corpus import stopwords

stop_words = stopwords.words(LANGUAGE)

In [8]:
def check_subset(label, description):
    label = label.lower()
    description = description.lower()
    
    clean_label = re.sub(r'\W+', ' ',label).split() # for german, only \W+
    clean_desc = re.sub(r'\W+', ' ', description).split()
    
    labels_sw_removed = [label_term for label_term in clean_label if label_term not in stop_words]
    desc_sw_removed = [desc_term for desc_term in clean_desc if desc_term not in stop_words]
    
    if len(desc_sw_removed) > 0:
        intersections = len(set(labels_sw_removed).intersection(desc_sw_removed))/len(desc_sw_removed) # remove stopwords, and also set a threshold for intersection before removal
    
        return intersections
    return 0
        

In [9]:
skill_pref_desc.columns = ['?a', 'prefLabel', 'desc']

In [10]:
skill_pref_desc['pref_desc_intesection'] = skill_pref_desc.parallel_apply(lambda x: check_subset(x.prefLabel, x.desc), axis=1)
skill_pref_desc['desc_pref_intesection'] = skill_pref_desc.parallel_apply(lambda x: check_subset(x.desc, x.prefLabel), axis=1)

In [11]:
skill_pref_desc.head()

,?a,prefLabel,desc,pref_desc_intesection,desc_pref_intesection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Lagerung von Rohstoffen und anderen Lebensmitt...,0.000000,0.000000
1,http://data.europa.eu/esco/skill/1a61a520-f4a5...,geschriebenes Irisch verstehen,Texte auf Irisch lesen und verstehen.,0.500000,0.666667
2,http://data.europa.eu/esco/skill/f6c10ed9-763a...,mit heißem Stickstoff arbeiten,"Arbeiten mit heißem Stickstoff, indem dieser d...",0.500000,1.000000
3,http://data.europa.eu/esco/skill/7e11b04d-c6b5...,Patienten/Patientinnen zur Verbesserung des Sp...,Beraten und Anleiten von Patienten in Technike...,0.250000,0.400000
4,http://data.europa.eu/esco/skill/5279200c-d6ed...,Forschungsergebnisse in die Musiktherapie inte...,Nutzung der aus verschiedenen wissenschaftlich...,0.058824,0.333333


In [12]:
100*len(skill_pref_desc[skill_pref_desc['pref_desc_intesection'] == 0])/skill_pref_desc.shape[0]

39.523022771352636

### ENGLISH: only 10.23% of the label data is not available in the term descriptions, and so we can use nearest neighbours using term description as the anchor and the other descriptions as from the search space


### GERMAN:  for german, the insterseciotn from preflabels into desc is 32.61%, still small, so also apply nn search to min negatives

In [13]:
# skill_pref_desc.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/positive_intersections.csv', sep='\t')

### Check statistics of term desc intersections in negative pairs

In [14]:
negatives_file = pd.read_csv(os.path.join(data_dir, '../', 'negative_100.csv'), sep='\t', index_col=[0])

In [15]:
negatives_file.rename(columns={'main_uri': '?a'}, inplace=True)

negatives_file_prefLabels = MemConcept_SkillsHier_prefLabel.merge(negatives_file, on=['?a'], how='left')
negatives_file_prefLabels.dropna(inplace=True)

negatives_file_prefLabels['negative_uris'] = negatives_file_prefLabels['negative_uris'].apply(lambda x: ast.literal_eval(x))

negatives_with_main_uri = negatives_file_prefLabels.explode('negative_uris')

negatives_with_main_uri.columns = ['main_uri', 'prefLabel', '?a']

In [16]:
skills_desc = skill_pref_desc[['?a', 'desc']]
skills_desc.rename(columns={'desc': 'negative_desc'}, inplace=True)

skills_desc.head()

/scratch/tmp.7262910.yakram/ipykernel_137919/2175954245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_desc.rename(columns={'desc': 'negative_desc'}, inplace=True)


,?a,negative_desc
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lagerung von Rohstoffen und anderen Lebensmitt...
1,http://data.europa.eu/esco/skill/1a61a520-f4a5...,Texte auf Irisch lesen und verstehen.
2,http://data.europa.eu/esco/skill/f6c10ed9-763a...,"Arbeiten mit heißem Stickstoff, indem dieser d..."
3,http://data.europa.eu/esco/skill/7e11b04d-c6b5...,Beraten und Anleiten von Patienten in Technike...
4,http://data.europa.eu/esco/skill/5279200c-d6ed...,Nutzung der aus verschiedenen wissenschaftlich...


In [17]:
negative_file_desc_merged = negatives_with_main_uri.merge(skills_desc, on=['?a'], how='left')

negative_file_desc_merged.dropna(inplace=True)

negative_file_desc_merged.head()

,main_uri,prefLabel,?a,negative_desc
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/815d1585-1706...,Behandeln von Holz mit verschiedenen Chemikali...
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5e433def-210d...,Vorschriftsmäßige Entsorgung von Abfall unter ...
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5d1a7c14-188d...,"Sicherstellen, dass die Reise reibungslos und ..."
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/8b6f2996-218b...,"Die Grundsätze, Bewertungstechniken und Behand..."
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/779f5db1-66db...,Überwachen der einzelnen Produktionsbudgets un...


In [18]:
negative_file_desc_merged['pref_desc_intersection'] = negative_file_desc_merged.parallel_apply(lambda x: check_subset(x.prefLabel, x.negative_desc), axis=1)
negative_file_desc_merged['desc_pref_intersection'] = negative_file_desc_merged.parallel_apply(lambda x: check_subset(x.negative_desc, x.prefLabel), axis=1)

In [19]:
negative_file_desc_merged.head()

,main_uri,prefLabel,?a,negative_desc,pref_desc_intersection,desc_pref_intersection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/815d1585-1706...,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.0
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5e433def-210d...,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.0
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5d1a7c14-188d...,"Sicherstellen, dass die Reise reibungslos und ...",0.0,0.0
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/8b6f2996-218b...,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.0
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/779f5db1-66db...,Überwachen der einzelnen Produktionsbudgets un...,0.0,0.0


In [20]:
# negative_file_desc_merged.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/negative_intersections.csv', sep='\t')

In [21]:
100*len(negative_file_desc_merged[negative_file_desc_merged['pref_desc_intersection'] == 0])/negative_file_desc_merged.shape[0]

98.84805238463285

In [22]:
negative_file_desc_merged['?a'].nunique()

13921

In [23]:
negative_file_desc_merged[negative_file_desc_merged['pref_desc_intersection'] > 0]['?a'].nunique(), negative_file_desc_merged[negative_file_desc_merged['pref_desc_intersection'] > 0]['?a'].shape

(7677, (16037,))

In [24]:
# raise

### merging original and the negative pairs with both their scores

In [25]:
negative_file_desc_merged.columns

Index(['main_uri', 'prefLabel', '?a', 'negative_desc',
       'pref_desc_intersection', 'desc_pref_intersection'],
      dtype='object')

In [26]:
skill_pref_desc.columns

Index(['?a', 'prefLabel', 'desc', 'pref_desc_intesection',
       'desc_pref_intesection'],
      dtype='object')

In [27]:
skill_pref_desc.columns = ['?a', 'prefLabel', 'desc_positive', 'positive_pref_desc_intesection',
       'positive_desc_pref_intesection']

In [28]:
negative_file_filtered = negative_file_desc_merged[['main_uri', 'prefLabel', 'negative_desc', 'pref_desc_intersection',
                                                   'desc_pref_intersection']]

negative_file_filtered.columns = ['?a', 'prefLabel', 'desc_negative', 'negative_pref_desc_intesection',
       'negative_desc_pref_intesection']

In [29]:
triplets_with_intersections_full = negative_file_filtered.merge(skill_pref_desc, on=['?a', 'prefLabel'], how='left')

In [30]:
triplets_with_intersections_full.head()

,?a,prefLabel,desc_negative,negative_pref_desc_intesection,negative_desc_pref_intesection,desc_positive,positive_pref_desc_intesection,positive_desc_pref_intesection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.0,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.0,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Sicherstellen, dass die Reise reibungslos und ...",0.0,0.0,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.0,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Überwachen der einzelnen Produktionsbudgets un...,0.0,0.0,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0


In [31]:
triplets_with_intersections_full.shape

(1392164, 8)

In [32]:
triplets_with_intersections_full.dropna(inplace=True)

In [33]:
# triplets_with_intersections_full.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/triplets_with_intersection_only.csv', sep='\t')

### get nearest neighbours for prefLabel and positive descriptions

In [34]:
model.to('cuda')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [35]:
negative_pref_data = negative_file_filtered[['?a', 'prefLabel', 'desc_negative']]

In [36]:
negative_pref_data_grouped = negative_pref_data.groupby(by=['?a','prefLabel'])['desc_negative'].apply(list).reset_index()

In [37]:
negative_pref_data_grouped.head()

,?a,prefLabel,desc_negative
0,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,"[Anwenden von Verfahren und Techniken, die gee..."
1,http://data.europa.eu/esco/isced-f/000,Allgemeine Bildungsprogramme und Qualifikation...,"[Vermessen des Platzes für eine Vorstellung, u..."
2,http://data.europa.eu/esco/isced-f/0000,Allgemeine Bildungsprogramme und Qualifikation...,[Erhebliche medizinische Probleme in einer Wei...
3,http://data.europa.eu/esco/isced-f/001,Grundlegende Bildungsprogramme und Qualifikati...,[Die angebotenen psychologischen Gesundheitsma...
4,http://data.europa.eu/esco/isced-f/0011,Grundlegende Bildungsprogramme und Qualifikati...,"[Die medizinische Behandlungsmethode, bei der ..."


In [38]:
def find_negatives_by_nn(main_uri, df_copy):
    
    df_copy_grouped = df_copy[df_copy['?a'] == main_uri]
    negative_corpus = df_copy_grouped['desc_negative'].iloc[0]
    main_term = df_copy_grouped['prefLabel'].unique().tolist()
    
    negative_embeddings = model.encode(negative_corpus, convert_to_tensor=True)
    query_embedding = model.encode(main_term, convert_to_tensor=True)
    
    hits = util.semantic_search(query_embedding, negative_embeddings, score_function=util.cos_sim, top_k=1000)
    
    return str(hits[0])

In [39]:
negative_pref_data_grouped['hits'] = negative_pref_data_grouped.progress_apply(lambda x: find_negatives_by_nn(x['?a'], negative_pref_data_grouped), axis=1)

100%|██████████| 14525/14525 [35:29<00:00,  6.82it/s]


In [40]:
negative_pref_data_grouped.head() 

,?a,prefLabel,desc_negative,hits
0,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,"[Anwenden von Verfahren und Techniken, die gee...","[{'corpus_id': 85, 'score': 0.4064350724220276..."
1,http://data.europa.eu/esco/isced-f/000,Allgemeine Bildungsprogramme und Qualifikation...,"[Vermessen des Platzes für eine Vorstellung, u...","[{'corpus_id': 9, 'score': 0.22099244594573975..."
2,http://data.europa.eu/esco/isced-f/0000,Allgemeine Bildungsprogramme und Qualifikation...,[Erhebliche medizinische Probleme in einer Wei...,"[{'corpus_id': 31, 'score': 0.3141812086105346..."
3,http://data.europa.eu/esco/isced-f/001,Grundlegende Bildungsprogramme und Qualifikati...,[Die angebotenen psychologischen Gesundheitsma...,"[{'corpus_id': 37, 'score': 0.6379895806312561..."
4,http://data.europa.eu/esco/isced-f/0011,Grundlegende Bildungsprogramme und Qualifikati...,"[Die medizinische Behandlungsmethode, bei der ...","[{'corpus_id': 75, 'score': 0.5975141525268555..."


In [41]:
negative_pref_data_grouped.shape

(14525, 4)

In [42]:
def append_scores(main_uri, df_large, all_data_combined):
    
    df = df_large[df_large['?a'] == main_uri]
    exploded_df = df.explode('desc_negative')
    
    hits_list = ast.literal_eval(exploded_df['hits'].iloc[0])
    hits_df = pd.DataFrame(hits_list)
    hits_df.sort_values(by='corpus_id', ascending=True, inplace=True)
    
    hits_df.reset_index(drop=True, inplace=True)
    exploded_df.reset_index(drop=True, inplace=True)
    
    new_df = pd.concat([exploded_df, hits_df[['score']]], axis='columns')
    
    all_data_combined.append(new_df)
    
    return all_data_combined

In [43]:
df_list = list()

scores_appended_df = negative_pref_data_grouped.progress_apply(
    lambda x: append_scores(x['?a'],negative_pref_data_grouped, df_list )
, axis=1)

100%|██████████| 14525/14525 [01:40<00:00, 143.86it/s]


In [44]:
len(df_list)

14525

In [45]:
negatives_pref_data_nn_scores = pd.DataFrame()

for df in tqdm(df_list):
    negatives_pref_data_nn_scores = pd.concat([negatives_pref_data_nn_scores, df])

100%|██████████| 14525/14525 [06:41<00:00, 36.20it/s]


In [46]:
negatives_pref_data_nn_scores.drop(columns='hits', inplace=True)

In [47]:
negatives_pref_data_nn_scores.head()

,?a,prefLabel,desc_negative,score
0,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,"Anwenden von Verfahren und Techniken, die geei...",0.045694
1,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,Vorstellung des Vorschlags von neuen Gesetzen ...,0.158892
2,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,"Anordnung des Inhalts eines Lagerbereichs, sod...",-0.055051
3,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,Kontrolle der Sicherheitsgurte von Fahrgeschäf...,0.032643
4,http://data.europa.eu/esco/isced-f/00,Allgemeine Bildungsprogramme und Qualifikationen,"Lesen und Analysieren von Fragebögen, Liefern ...",0.099613


In [48]:
negatives_pref_data_nn_scores.columns= ['?a', 'prefLabel', 'desc_negative', 'negative_desc_nn_score']

In [49]:
negative_file_filtered.shape, negatives_pref_data_nn_scores.shape

((1392164, 5), (1392164, 4))

In [50]:
negative_pref_desc_with_scores = negative_file_filtered.merge(negatives_pref_data_nn_scores,
                                                              on=['?a', 'prefLabel', 'desc_negative'], how='left')

In [51]:
negative_pref_desc_with_scores.head()

,?a,prefLabel,desc_negative,negative_pref_desc_intesection,negative_desc_pref_intesection,negative_desc_nn_score
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.0,0.183325
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.0,0.376603
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Sicherstellen, dass die Reise reibungslos und ...",0.0,0.0,-0.014479
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.0,0.168873
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Überwachen der einzelnen Produktionsbudgets un...,0.0,0.0,-0.044176


In [52]:
triplets_with_nn_scores_full = negative_pref_desc_with_scores.merge(skill_pref_desc, on=['?a', 'prefLabel'], how='left')

In [53]:
triplets_with_nn_scores_full.head()

,?a,prefLabel,desc_negative,negative_pref_desc_intesection,negative_desc_pref_intesection,negative_desc_nn_score,desc_positive,positive_pref_desc_intesection,positive_desc_pref_intesection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.0,0.183325,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.0,0.376603,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Sicherstellen, dass die Reise reibungslos und ...",0.0,0.0,-0.014479,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.0,0.168873,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Überwachen der einzelnen Produktionsbudgets un...,0.0,0.0,-0.044176,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0


In [54]:
# triplets_with_nn_scores_full.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/triplets_with_nn_scores_full.csv', sep='\t')

In [55]:
triplets_with_nn_scores_full['?a'].nunique()

14525

In [56]:
triplets_with_nn_scores_full[triplets_with_nn_scores_full['negative_pref_desc_intesection'] > 0]['?a'].nunique()

7002

In [57]:
100*11509/ 14525

79.23580034423408

### a total of 79.23% of the nearest neightbours have an intersection term as well, let's now select the instersection terms based on nn criteria

In [58]:
triplets_with_nn_scores_full.head()

,?a,prefLabel,desc_negative,negative_pref_desc_intesection,negative_desc_pref_intesection,negative_desc_nn_score,desc_positive,positive_pref_desc_intesection,positive_desc_pref_intesection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.0,0.183325,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.0,0.376603,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Sicherstellen, dass die Reise reibungslos und ...",0.0,0.0,-0.014479,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.0,0.168873,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Überwachen der einzelnen Produktionsbudgets un...,0.0,0.0,-0.044176,Lagerung von Rohstoffen und anderen Lebensmitt...,0.0,0.0


In [59]:
triplet_data_with_scores = triplets_with_nn_scores_full[['?a', 'prefLabel', 'desc_negative', 
                                                         'negative_pref_desc_intesection', 'negative_desc_nn_score']]

In [60]:
triplet_data_with_scores.head()

,?a,prefLabel,desc_negative,negative_pref_desc_intesection,negative_desc_nn_score
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Behandeln von Holz mit verschiedenen Chemikali...,0.0,0.183325
1,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Vorschriftsmäßige Entsorgung von Abfall unter ...,0.0,0.376603
2,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Sicherstellen, dass die Reise reibungslos und ...",0.0,-0.014479
3,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,"Die Grundsätze, Bewertungstechniken und Behand...",0.0,0.168873
4,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Überwachen der einzelnen Produktionsbudgets un...,0.0,-0.044176


In [61]:
def create_grouped_data(df, column_name):
    
    df_subset = df[['?a', column_name]]
    
    df_subset_grouped = df_subset.groupby(by=['?a'])[column_name].apply(list).reset_index()
    
    return df_subset_grouped

In [62]:
triplet_grouped_desc = create_grouped_data(triplet_data_with_scores, 'desc_negative')
triplet_grouped_negative_pref_desc_score = create_grouped_data(triplet_data_with_scores, 'negative_pref_desc_intesection')
triplet_grouped_negative_desc_nn_score = create_grouped_data(triplet_data_with_scores, 'negative_desc_nn_score')

In [63]:
triplet_data_appended = triplet_grouped_desc.merge(
    triplet_grouped_negative_pref_desc_score, on=['?a'], how='left').merge(
    triplet_grouped_negative_desc_nn_score, on=['?a'], how='left')

In [64]:
triplet_data_appended.head()

,?a,desc_negative,negative_pref_desc_intesection,negative_desc_nn_score
0,http://data.europa.eu/esco/isced-f/00,"[Anwenden von Verfahren und Techniken, die gee...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04569447040557861, 0.15889166295528412, -0...."
1,http://data.europa.eu/esco/isced-f/000,"[Vermessen des Platzes für eine Vorstellung, u...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.04008367657661438, -0.07715119421482086, 0..."
2,http://data.europa.eu/esco/isced-f/0000,[Erhebliche medizinische Probleme in einer Wei...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07257924228906631, 0.04098954796791077, -0...."
3,http://data.europa.eu/esco/isced-f/001,[Die angebotenen psychologischen Gesundheitsma...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2016340047121048, 0.013529770076274872, 0.0..."
4,http://data.europa.eu/esco/isced-f/0011,"[Die medizinische Behandlungsmethode, bei der ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.009830817580223083, 0.4399043321609497, -0..."


In [65]:
# triplet_data_appended.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/triplet_data_without_negatives.csv', sep='\t')

In [66]:
def compute_hard_negatives(main_uri, df):
    
    df_filtered = df[df['?a'] == main_uri]
    df_subset = df_filtered.explode(['desc_negative','negative_pref_desc_intesection', 'negative_desc_nn_score'])
    df_subset.reset_index(drop=True, inplace=True)
    
    df_intersected = df_subset[df_subset['negative_pref_desc_intesection'] > 0]
    
    if len(df_intersected) == 0:
        df_subset.sort_values(by='negative_desc_nn_score', ascending=False, inplace=True)
        return df_subset['desc_negative'].iloc[0]
    else:
        df_intersected.sort_values(by='negative_desc_nn_score', ascending=False, inplace=True)

        return df_intersected['desc_negative'].iloc[0]

In [67]:
triplet_data_appended['hard_negative'] = triplet_data_appended.parallel_apply(
    lambda x: compute_hard_negatives(x['?a'], triplet_data_appended), axis=1
                                                                                             )

In [68]:
# triplet_data_appended.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/triplets_hard_negatives.csv', sep='\t')

# creating final classification_data

In [69]:
triplet_data_appended.head()

,?a,desc_negative,negative_pref_desc_intesection,negative_desc_nn_score,hard_negative
0,http://data.europa.eu/esco/isced-f/00,"[Anwenden von Verfahren und Techniken, die gee...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04569447040557861, 0.15889166295528412, -0....","Bewertung der Vorkenntnisse von Studierenden, ..."
1,http://data.europa.eu/esco/isced-f/000,"[Vermessen des Platzes für eine Vorstellung, u...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.04008367657661438, -0.07715119421482086, 0...",Schüler in der Regel zu Freizeitzwecken in der...
2,http://data.europa.eu/esco/isced-f/0000,[Erhebliche medizinische Probleme in einer Wei...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07257924228906631, 0.04098954796791077, -0....","Planung, Durchführung und Beaufsichtigung von ..."
3,http://data.europa.eu/esco/isced-f/001,[Die angebotenen psychologischen Gesundheitsma...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2016340047121048, 0.013529770076274872, 0.0...",Beaufsichtigen von Auszubildenden in den sozia...
4,http://data.europa.eu/esco/isced-f/0011,"[Die medizinische Behandlungsmethode, bei der ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.009830817580223083, 0.4399043321609497, -0...","Arbeiten in einer Berufsschule, die praktische..."


In [70]:
skill_pref_desc.head()

,?a,prefLabel,desc_positive,positive_pref_desc_intesection,positive_desc_pref_intesection
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Lagerung von Rohstoffen und anderen Lebensmitt...,0.000000,0.000000
1,http://data.europa.eu/esco/skill/1a61a520-f4a5...,geschriebenes Irisch verstehen,Texte auf Irisch lesen und verstehen.,0.500000,0.666667
2,http://data.europa.eu/esco/skill/f6c10ed9-763a...,mit heißem Stickstoff arbeiten,"Arbeiten mit heißem Stickstoff, indem dieser d...",0.500000,1.000000
3,http://data.europa.eu/esco/skill/7e11b04d-c6b5...,Patienten/Patientinnen zur Verbesserung des Sp...,Beraten und Anleiten von Patienten in Technike...,0.250000,0.400000
4,http://data.europa.eu/esco/skill/5279200c-d6ed...,Forschungsergebnisse in die Musiktherapie inte...,Nutzung der aus verschiedenen wissenschaftlich...,0.058824,0.333333


In [71]:
negative_subset = triplet_data_appended[['?a', 'hard_negative']]

In [72]:
positive_subset = skill_pref_desc[['?a', 'prefLabel', 'desc_positive']]

In [73]:
positive_subset.head()

,?a,prefLabel,desc_positive
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,Lagerung von Rohstoffen und anderen Lebensmitt...
1,http://data.europa.eu/esco/skill/1a61a520-f4a5...,geschriebenes Irisch verstehen,Texte auf Irisch lesen und verstehen.
2,http://data.europa.eu/esco/skill/f6c10ed9-763a...,mit heißem Stickstoff arbeiten,"Arbeiten mit heißem Stickstoff, indem dieser d..."
3,http://data.europa.eu/esco/skill/7e11b04d-c6b5...,Patienten/Patientinnen zur Verbesserung des Sp...,Beraten und Anleiten von Patienten in Technike...
4,http://data.europa.eu/esco/skill/5279200c-d6ed...,Forschungsergebnisse in die Musiktherapie inte...,Nutzung der aus verschiedenen wissenschaftlich...


In [74]:
negative_subset.head()

,?a,hard_negative
0,http://data.europa.eu/esco/isced-f/00,"Bewertung der Vorkenntnisse von Studierenden, ..."
1,http://data.europa.eu/esco/isced-f/000,Schüler in der Regel zu Freizeitzwecken in der...
2,http://data.europa.eu/esco/isced-f/0000,"Planung, Durchführung und Beaufsichtigung von ..."
3,http://data.europa.eu/esco/isced-f/001,Beaufsichtigen von Auszubildenden in den sozia...
4,http://data.europa.eu/esco/isced-f/0011,"Arbeiten in einer Berufsschule, die praktische..."


In [75]:
classficiation_data_triplets = negative_subset.merge(positive_subset, on=['?a'], how='left')

In [76]:
classficiation_data_triplets.shape

(14525, 4)

In [77]:
classficiation_data_triplets.head()

,?a,hard_negative,prefLabel,desc_positive
0,http://data.europa.eu/esco/isced-f/00,"Bewertung der Vorkenntnisse von Studierenden, ...",NaN,NaN
1,http://data.europa.eu/esco/isced-f/000,Schüler in der Regel zu Freizeitzwecken in der...,NaN,NaN
2,http://data.europa.eu/esco/isced-f/0000,"Planung, Durchführung und Beaufsichtigung von ...",NaN,NaN
3,http://data.europa.eu/esco/isced-f/001,Beaufsichtigen von Auszubildenden in den sozia...,NaN,NaN
4,http://data.europa.eu/esco/isced-f/0011,"Arbeiten in einer Berufsschule, die praktische...",NaN,NaN


In [78]:
positive_classification_data = classficiation_data_triplets[['?a', 'prefLabel', 'desc_positive']]
negative_classification_data = classficiation_data_triplets[['?a', 'prefLabel', 'hard_negative']]

In [79]:
positive_classification_data.dropna(inplace=True)
negative_classification_data.dropna(inplace=True)

/scratch/tmp.7262910.yakram/ipykernel_137919/4189035163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_classification_data.dropna(inplace=True)
/scratch/tmp.7262910.yakram/ipykernel_137919/4189035163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_classification_data.dropna(inplace=True)


In [80]:
positive_classification_data.head()

,?a,prefLabel,desc_positive
220,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...
221,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...
222,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ..."
223,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
224,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...


In [81]:
negative_classification_data.head()

,?a,prefLabel,hard_negative
220,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Die verschiedenen Technologien für Tonprodukti...
221,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Ausstellung von Bescheinigungen für Tiergesund...
222,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,Praktiken in Bezug auf die sichere Reinigung v...
223,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,Durchführen der Reparatur und routinemäßigen W...
224,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Erweiterung des Dienstleistungsangebots für Ku...


In [82]:
positive_classification_data['labels'] = 1
negative_classification_data['labels'] = 0

/scratch/tmp.7262910.yakram/ipykernel_137919/1617624918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_classification_data['labels'] = 1
/scratch/tmp.7262910.yakram/ipykernel_137919/1617624918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_classification_data['labels'] = 0


In [83]:
positive_classification_data.columns = ['?a', 'term1', 'term2', 'labels']
negative_classification_data.columns = ['?a', 'term1', 'term2', 'labels']

In [84]:
classification_data_full = pd.concat([positive_classification_data, negative_classification_data])

In [85]:
classification_data_full.reset_index(drop=True, inplace=True)

In [86]:
classification_data_full.head()

,?a,term1,term2,labels
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...,1
1,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...,1
2,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ...",1
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ...",1
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...,1


In [87]:
# classification_data_full.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/classification_term_desc.csv', sep='\t')

### shuffle classification data

In [88]:
shuffled_classification_data = classification_data_full.sample(frac=1)

In [89]:
first_half = shuffled_classification_data[: shuffled_classification_data.shape[0]]
second_half = shuffled_classification_data[shuffled_classification_data.shape[0]:]

In [90]:
first_half_shuffled = first_half[['?a', 'term2', 'term1', 'labels']]

In [91]:
first_half_shuffled.head()

,?a,term2,term1,labels
23537,http://data.europa.eu/esco/skill/b0275474-2d6a...,"Die verschiedenen Dienste wie Liniendienste, S...",Schiffs-Steuerungsvorrichtungen warten,0
1673,http://data.europa.eu/esco/skill/1f4a4458-af5e...,Kräne und Krananlagen reparieren und bei Bedar...,Krananlagen reparieren,1
26149,http://data.europa.eu/esco/skill/e18ee06c-7a0f...,"Hören, Verstehen und Tippen von Inhalt aus Aud...",gesprochenes Okzitanisch verstehen,0
18787,http://data.europa.eu/esco/skill/58d8e348-8bbc...,"Finanzbereich, der sich mit der Analyse prakti...",Anlageberatung leisten,0
24399,http://data.europa.eu/esco/skill/c0f51665-5a3c...,"Maschinen, die für die Baustoffhandhabung in a...",Maschinen für den Möbelbau reparieren,0


In [92]:
classification_data_shuffled = pd.concat([first_half_shuffled, second_half])

classification_data_shuffled.reset_index(inplace=True, drop=True)

In [93]:
classification_data_shuffled.to_csv(f'/cluster/scratch/yakram/sbert-copy/turtle_files/classification_data/{model_dir_language}/shuffled_classification_data_desc.csv', sep='\t')